Function definitions for nearest neighbor, savings, and 2-opt algorithms.

Function definitions for Nearest neighbor algorithm

In [ ]:
def nearest_neighbor(nodes, origin, d):
    '''
    Constructs a TSP solution using the nearest neighbor algorithm, NNH,
    for a given set of nodes, the associated pairwise distance matrix-d,
    and the origin.
    '''
    # Tour should start at the origin
    tour = [origin]

    # Initialize the tour length
    tour_length = 0

    # If the origin is not in nodes, add it to nodes
    if origin not in nodes:
        nodes.append(origin)

    # Nearest neighbor search until all nodes are visited
    while len(tour) < len(nodes):
        dist, next_node = min((d[tour[-1]][i], i) for i in nodes if i not in tour)
        tour_length += dist
        tour.append(next_node)
        print('Added', next_node, 'to the tour!')

    # Tour should end at the origin
    tour_length += d[tour[-1]][origin]
    tour.append(origin)

    # Round the result to 2 decimals to avoid floating point representation errors
    tour_length = round(tour_length, 2)

    # Return the resulting tour and its length as a tuple
    return tour, tour_length

Function definitions for nearest Savings algorithm

In [ ]:
def savings(nodes, origin, d):
    '''
    Constructs a TSP solution using the savings method for a given set/list of
    nodes, their pairwise distances-d, and the origin.
    '''
    # Set of customer nodes (i.e. nodes other than the origin)
    customers = {i for i in nodes if i != origin}

    # Initialize out-and-back tours from the origin to every other node
    tours = {(i,i): [origin, i, origin] for i in customers}

    # Compute savings
    savings = {(i, j): round(d[i][origin] + d[origin][j] - d[i][j], 2)
               for i in customers for j in customers if j != i}

    # Define a priority queue dictionary to get a pair of nodes (i,j) which yields
    # the maximum savings
    #pq = pqdict(savings, reverse = True)
    sorted_savings = sorted(savings.items(), key=lambda item: item[1])
    # Merge subtours until obtaining a TSP tour
    while len(tours) > 1:
        A = sorted_savings.pop()
        i = A[0][0]
        j = A[0][1]
        print((i, j))
        # Outer loop
        break_outer = False
        for t1 in tours:
            for t2 in tours.keys()-{t1}:
                if t1[1] == i and t2[0] == j:
                    print('Merging', tours[t1], 'and', tours[t2])
                    tours[(t1[0], t2[1])] = tours[t1][:-1] + tours[t2][1:]
                    del tours[t1], tours[t2]
                    print(tours)
                    break_outer = True
                    break
            if break_outer:
                break
        else:
            print('No merging opportunities can be found for', (i,j))

    # Final tours dictionary (involves a single tour, which is the TSP tour)
    print(tours)

    # Compute tour length
    tour_length = 0
    for tour in tours.values():
        for i in range(len(tour)-1):
            tour_length += d[tour[i]][tour[i+1]]

    # Round the result to 2 decimals to avoid floating point representation errors
    tour_length = round(tour_length, 2)

    # Return the resulting tour and its length as a tuple
    return tour, tour_length


Function definitions for 2-opt algorithm

In [ ]:
def two_opt(tour, tour_length, d):
    '''
    Improves a given TSP solution using the 2-opt algorithm. Note: This function
    applies 2opt correctly only when the distance matrix is symmetric. In case
    of asymmetric distances, one needs to update the cost difference calculation
    incurred by swapping.
    '''
    current_tour, current_tour_length = tour, tour_length
    best_tour, best_tour_length = current_tour, current_tour_length
    solution_improved = True

    while solution_improved:
        print()
        print('Attempting to improve the tour', current_tour,
              'with length', current_tour_length)
        solution_improved = False

        for i in range(1, len(current_tour)-2):
            for j in range(i+1, len(current_tour)-1):
                difference = round((d[current_tour[i-1]][current_tour[j]]
                                  + d[current_tour[i]][current_tour[j+1]]
                                  - d[current_tour[i-1]][current_tour[i]]
                                  - d[current_tour[j]][current_tour[j+1]]), 2)

                print('Cost difference due to swapping', current_tour[i], 'and',
                      current_tour[j], 'is:', difference)

                if current_tour_length + difference < best_tour_length:
                    print('Found an improving move! Updating the best tour...')

                    best_tour = current_tour[:i] + list(reversed(current_tour[i:j+1])) + current_tour[j+1:]
                    best_tour_length = round(current_tour_length + difference, 2)

                    print('Improved tour is:', best_tour, 'with length',
                          best_tour_length)

                    solution_improved = True

        current_tour, current_tour_length = best_tour, best_tour_length

    # Return the resulting tour and its length as a tuple
    return best_tour, best_tour_length

Function for Relocate Algorithm

In [ ]:
def relocate(route, depot, d):
    best_route = route.copy()
    best_cost = sum(d[route[i]][route[i+1]] for i in range(len(route)-1))
    improved = True

    while improved:
        improved = False
        for i in range(1, len(best_route) - 1):
            for j in range(1, len(best_route) - 1):
                if i == j:
                    continue
                new_route = best_route.copy()
                node = new_route.pop(i)
                new_route.insert(j, node)
                new_cost = sum(d[new_route[k]][new_route[k+1]] for k in range(len(new_route)-1))
                if new_cost < best_cost:
                    best_route = new_route
                    best_cost = new_cost
                    improved = True

    return best_route, best_cost

In [ ]:
import networkx as nx
from itertools import combinations
import numpy as np

def christofides(d):
    """
    Christofides TSP approximation algorithm.
    Input: d - distance matrix (NumPy array)
    Output: a TSP tour visiting all nodes once and returning to start
    """
    G = nx.Graph()
    n = len(d)

    # Step 1: Add all edges to complete graph
    for i in range(n):
        for j in range(i + 1, n):
            G.add_edge(i, j, weight=d[i][j])

    # Step 2: Compute MST
    T = nx.minimum_spanning_tree(G)

    # Step 3: Find odd-degree nodes in MST
    odd_nodes = [v for v in T.nodes if T.degree[v] % 2 == 1]

    # Step 4: Minimum weight perfect matching among odd nodes
    M = nx.Graph()
    for u, v in combinations(odd_nodes, 2):
        M.add_edge(u, v, weight=d[u][v])
    matching = nx.algorithms.matching.min_weight_matching(M)

    # Step 5: Combine MST and Matching to form multigraph
    multigraph = nx.MultiGraph()
    multigraph.add_edges_from(T.edges(data=True))
    multigraph.add_edges_from(((u, v, {'weight': d[u][v]}) for u, v in matching))

    # Step 6: Find Eulerian circuit
    euler = list(nx.eulerian_circuit(multigraph, source=0))

    # Step 7: Shortcut to create Hamiltonian cycle
    visited = set()
    tour = []
    for u, v in euler:
        if u not in visited:
            visited.add(u)
            tour.append(u)
        if len(visited) == n:
            break
    tour.append(0)  # return to depot

    return tour
